# Lesson 11: Pagination 
---
Intro: Let's look into pagination where we can limit a certain amount of posts per page.

# Review
---

1. What are the differences between class-based views and function-based views?
2. What do all the generic views (DetailView, ListView, UpdatView, etc) have in common?
3. How do you create a new URL path?
3. How's this course going for you?

# Concept 1: Pagination
---


## Objective
Our blog site will eventually contain many posts and we need a way to contain them. With pagination, we can limit the number of posts per page. 
## Pagination
1. Look at the ListView's documentation [here](https://docs.djangoproject.com/en/3.1/topics/pagination/#paginating-a-listview). 
2. So you saw that we can paginate our posts using the `paginate_by` variable. Go to the blog's views.py page and under the PostListView class, include this `paginate_by = 2`.
* This limits the amount of posts per page by 2.
3. Go to the browser and check this out. Create more posts with different users to see the difference.
4. Now we need a way to switch between different pages. You can actually do this manually by going to `localhost:8000/?page=2`
5. Go to `home.html` under `blog/templates/blog`. We will edit this file because this is where our posts are being rendered.
6. Between `endfor` and `endblock content` include this if statement:
```
{% if is_paginated %}
{% endif %}
```
* By giving the ListView a paginated_by variable, this automatically sets the `is_paginated` to True thanks to Django.
7. Now in between that if statement, create this nested if statement:
```
{% if page_obj.has_previous %}
{% endif %}
```
* So overall, first it checks if the ListView is paginated then it checks if the current page has a previous page. (i.e You're on page 2. The current page has a previous page at page 1). If you're wondering where `page_obj` came from, by including `paginate_by` in the ListView, Django enables us to use that variable.
8. In between, the nested if statement include this:
```
<a class="btn btn-outline-info mb-4" href="?page=1">First</a>
<a class="btn btn-outline-info mb-4" href="?page={{ page_obj.previous_page_number }}">Previous</a>
```
* The first line will provide a hyperlinked button that will always go to the first page.
* The second line will provide a hyperlinked button that will always go to the previous page.
9. Now we want to display a range of pages. For example, if we are on page 5, there will be buttons for pages 3 and 4 and page 6 and 7. This will change depending on the current page.
10. Below the `if page_obj.has_previous ` statement, let's create a for loop that iterates through all the pages.
```
{% for num in page_obj.paginator.page_range %}
{% endfor %}
```
* Iterate through a page object (the ListView) with a paginator (object containing all the posts) that states the range (size of the list).
11. Next, in the for loop we will have an if and an else if statement to see which pages to render for the buttons. Type it out.
So our first if statement looks like this:
```
{% if page_obj.number == num %}
      <a class="btn btn-info mb-4" href="?page={{ num }}">{{ num }}</a>
{% endif %}
```
* This simply checks if the page we are on matches with num. If it is, provide the href with the current number and display a button with the current number as well. We also made this button a solid color by omitting `outline`.
12. Now for the else-if statement, we provide the page range. Type this out so you can understand it more.
```
{% elif num > page_obj.number|add:'-3' and num < page_obj.number|add:'3' %}
        <a class="btn btn-outline-info mb-4" href="?page={{ num }}">{{ num }}</a>
```
* This prints numbers surrounding the current page. Check if num is greater than the current page and within 3 previous, then render the buttons. Do this similarly when num is less than the current page. The `|` (pipe) filters our command.
13. Outside the for loop, let's display buttons for the next page numbers. This if statement is very similar to the has_previous if statement
```
{% if page_obj.has_next %}
        <a class="btn btn-outline-info mb-4" href="?page={{ page_obj.next_page_number }}">Next</a>
        <a class="btn btn-outline-info mb-4" href="?page={{ page_obj.paginator.num_pages }}">Last</a>
{% endif %}
```
* First, it checks if the page has a page that follows. If so, it displays the next page number as well as the very last page using `{ page_obj.paginator.num_pages }`
14. Let's check it out on the browser. Flip through the pages.
15. Now let's change the `paginate_by` variable to something bigger like 5 to display more posts. This is in the views.py file.
16. Overall, the code should look like this. Explain what each line does.

In [ ]:
{% if is_paginated %}

{% if page_obj.has_previous %}
  <a class="btn btn-outline-info mb-4" href="?page=1">First</a>
  <a class="btn btn-outline-info mb-4" href="?page={{ page_obj.previous_page_number }}">Previous</a>
{% endif %}

{% for num in page_obj.paginator.page_range %}
  {% if page_obj.number == num %}
    <a class="btn btn-info mb-4" href="?page={{ num }}">{{ num }}</a>
  {% elif num > page_obj.number|add:'-3' and num < page_obj.number|add:'3' %}
    <a class="btn btn-outline-info mb-4" href="?page={{ num }}">{{ num }}</a>
  {% endif %}
{% endfor %}

{% if page_obj.has_next %}
  <a class="btn btn-outline-info mb-4" href="?page={{ page_obj.next_page_number }}">Next</a>
  <a class="btn btn-outline-info mb-4" href="?page={{ page_obj.paginator.num_pages }}">Last</a>
{% endif %}

{% endif %}

## DIY:
---

1. Explain the `paginate_by` variable.
2. What does the pipe do?
3. What's the purpose of the for loop?

# Concept 2: User Pagination
---


## Objective
If you look at the home page, the user name has a dead link and it doesn't go anywhere. When you click on the user, you should see how many posts that user wrote. Let's build a route that displays a specific user's posts.
## User Pagination
17. Go to `views.py` under `blog/templates/blog`. We're going to make a separate ListView that holds an individual user's posts. This is very similar to PostListView so simply copy the `PostListView` and paste it underneath it. Change the function name to `UserPostListView`. Change the template_name to `'blog/user_posts.html'`. We will create this in a bit.
18. Now let's modify this so we can filter the list of posts to a single user. First let's import some methods. 
```
from django.shortcuts import get_object_or_404
```
* Note: you have shortcuts already imported. `get_object_or_404` simply means to get the object or to display a 404 error. 

```
from django.contrib.auth.models import User
```
* Since we need to access a specific user, we access the user model from our models class.

19. Inside the class, create a function like this:
```
def get_queryset(self):
```
* It determines the list of objects that you want to display. By default, it gives you all for the model you specify. By overriding this method you can extend or completely replace this logic.

20. Now in the `get_queryset` method type in this:
```
user = get_object_or_404(User, username=self.kwargs.get('username'))
```

* A user variable is passed in. If the user doesn't exist, then this returns a 404 error page.

21. Next, return all the posts filtered by the author and the date posted. 
```
return Post.objects.filter(author=user).order_by('date_posted')
```

22. Just a side note: since we are overriding the query (or list of post) this ListView is managing, our ordering will also be overwritten. So delete `ordering = ['-date_posted']`. 

## User Path for Pagination

23. Open the blog's urls.py page and import `UserPostListView` from `.views`. Since we have `.views` simply add it to the list of imports.
24. Now create a path for a specific user. Type this out so you can understand this line.
```
path('user/<str:username>', UserPostListView.as_view(), name='user-posts'),
```
* So similar to the integer primary key, we have a username primary key that will always be a string. 
* > Answer this: why do we have `as_view()`?
25. Let's create the `user_posts.html` template that we specified in the UserPostListView.
26. Under `blog/templates/blog` name it `user_posts.html`. This will be very similar to the `home.html` page so copy the code from `home.html` and paste it in. This presents all the posts as well as providing pagination.
27. So once we click on the user's name, it filters all the posts. We also want a title stating that the viewable posts are from the specific user. So, right below `{% block content %}` add the following:
```
<h1 class="mb-3">Posts by {{ view.kwargs.username }} ({{ page_obj.paginator.count }})</h1>
```
* Provide an H1 tag that says `Posts by Tomas (18)`. The kwargs value retrieves the username from the URL. This also prints out the total number of posts (or objects) from this user.

28. Remember when we clicked on the user's name and it didn't go anywhere? Now let's fix that. Find the a tag with `{{ post.author }}`. Change the href to this:
```
href="{% url 'user-posts' post.author.username %}"
```
* This simply gets the url for user-posts for the specific user.

29. Let's do this as well for the home template. So go to `home.html` and find `{{ post.author }}` and change the href to this:
```
href="{% url 'user-posts' post.author.username %}"
```
30. Let's change the same thing in `post_detail.html`. Find `{{ object.author }}` and change the href to this:
```
href="{% url 'user-posts' object.author.username %}"
```
* Note that this page uses objects. 
31. Let's open this in the browser and click on a user's name.
32. Now test for a bad user. Go to `localhost:8000/user/BadUser`. You should get a 404 error.


## DIY:
---

1. What's the purpose of `get_object_or_404`?
2. Explain `'user/<str:username>'` in `path('user/<str:username>', UserPostListView.as_view(), name='user-posts'),`
3. When using the templating engine with hrefs, what is the format used? Here is an example `href="{% url 'user-posts' post.author.username %}"`

# Summary:
---



1. Why do we need pagination in our website?
1. Explain the `paginate_by` variable.
2. What does the pipe do?
3. What's the purpose of `get_object_or_404`?


# Homework:
---



1. Continue the blackjack game.
2. If you're finished, now combine the war game and blackjack. Let the user decide which game to play.

# Notes on homework:
---

I will check in on Thursday,  through email to check on your progress. Respond with any questions you might have. Otherwise, a simple “all good” is appropriate if you have no questions or comments. 

You will need to upload your coding homework assignments to GitHub.
1. In gitbash, change directories to the homework directory: tomas_python/homework
* TIP: use ‘cd’ to change directories
* Use ‘cd ..’ to return to the previous directory
* Use ‘pwd’ to show full pathname of the current working directory 
* Use ‘ls’ to list all your directories
2. Once you’re in that directory, type in ‘git pull’
* This ensures you have all updated files
* If there is an error involved, email me immediately so we can try resolving it.
* Otherwise, type your code below and we’ll resolve issues next class
3. To create a new file, type in ‘touch hw01.py’ or the appropriate file name
* ‘Touch’ creates a new file
4. Open up the python file and start coding!

Note: Become familiar with these actions. This is essentially what happens in the backend when you right-click and create a new folder/file!
